In [30]:
%run 01_optimize_functions.py

In [32]:
# =============================================================================
#### start 
# =============================================================================

with tf.device("/gpu:0"): #or use cpu if you like
    
    time1 = datetime.datetime.now()
    basedir = './' #define working directory
    os.chdir(basedir)

    for pp in range(118): # loop all 118 sites
        time_single = datetime.datetime.now()
        seed(1)
        tf.random.set_seed(1)
        
        pathGW = "Data/GWData"
        pathHYRAS = "Data/HYRAS"
        pathconnect = "/"
        
        skip = True

# =============================================================================
#### parameter bounds and optimizer
# =============================================================================
        pbounds = {'pp': (pp,pp),
                   'seqlength': (1, 52), 
                   'densesize': (1, 256),
                   'batchsize': (16, 256),
                   'filters': (1, 256)} #constrained optimization technique, so you must specify the minimum and maximum values that can be probed for each parameter
        
        optimizer = BayesianOptimization(
            f= bayesOpt_function, #optimized function
            pbounds=pbounds, #parameter bounds
            random_state=1, 
            verbose = 0 # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent, verbose = 2 prints everything
            )
            
        logger = newJSONLogger(path="./logs_CNN_"+str(pp)+".json")
        optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
        
        optimizer.maximize(
                init_points=20, #steps of random exploration (random starting points before bayesopt(?))
                n_iter=0, # steps of bayesian optimization
             #   acq="ei",# ei  = expected improvmenet (probably the most common acquisition function) 
            #    xi=0.05  #  Prefer exploitation (xi=0.0) / Prefer exploration (xi=0.1)
                )
        
        counter1 = 60
        counter2 = 15
        counter3 = 150
            
        # optimize while improvement during last 10 steps
        current_step = len(optimizer.res)
        beststep = False
        step = -1
        while not beststep:
            step = step + 1
            beststep = optimizer.res[step] == optimizer.max #search for best iteration
    
        
        while current_step < counter1:
                current_step = len(optimizer.res)
                beststep = False
                step = -1
                while not beststep:
                    step = step + 1
                    beststep = optimizer.res[step] == optimizer.max
                print("\nbeststep {}, current step {}".format(step+1, current_step+1))
                optimizer.maximize(
                    init_points=0, #steps of random exploration (random starting points before bayesopt(?))
                    n_iter=1, # steps of bayesian optimization
                 #   acq="ei",# ei  = expected improvmenet (probably the most common acquisition function) 
                    xi=0.05  #  Prefer exploitation (xi=0.0) / Prefer exploration (xi=0.1)
                    )
        
        while (step + counter2 > current_step and current_step < counter3):
                current_step = len(optimizer.res)
                beststep = False
                step = -1
                while not beststep:
                    step = step + 1
                    beststep = optimizer.res[step] == optimizer.max
                    
                print("\nbeststep {}, current step {}".format(step+1, current_step+1))
                optimizer.maximize(
                    init_points=0, #steps of random exploration (random starting points before bayesopt(?))
                    n_iter=1, # steps of bayesian optimization
                    acq="ei",# ei  = expected improvmenet (probably the most common acquisition function) 
                    xi=0.05  #  Prefer exploitation (xi=0.0) / Prefer exploration (xi=0.1)
                    )
            
        print("\nBEST:\t{}".format(optimizer.max))

        #get best values from optimizer
        densesize_int = int(optimizer.max.get("params").get("densesize"))
        seqlength_int = int(optimizer.max.get("params").get("seqlength"))
        batchsize_int = int(optimizer.max.get("params").get("batchsize"))
        filters_int = int(optimizer.max.get("params").get("filters"))
        
        #run test set simulations
        t1_test = datetime.datetime.now()
        scores, TestData, sim, obs, inimax, sim_members, Well_ID, errors, sim_members_uncertainty, sim_uncertainty = simulate_testset(pp, densesize_int, seqlength_int, batchsize_int, filters_int)

        t2_test = datetime.datetime.now()
        f = open('./timelog_CNN_'+Well_ID+'.txt', "w")
        print("Time [s] for Test-Eval (10 inis)\n{}\n".format(t2_test-t1_test), file = f)

KeyError: 'GWL'